In [ ]:
import os

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from PIL import Image

import pickle
import numpy as np

import sharp
import h1data
import bq

In [ ]:
fname = "data/20230106_CaptureDL_melbourne_2023_01_06T23_50_06"
#fname = "data/20221010_CaptureDL_tampa_2022_10_08T15_39_46"
#fname = "data/20221027_CaptureDL_bangladesh_2022_10_26T04_02_54"
#fname = "data/20221205_CaptureDL_florida_2022_12_02T15_16_12"
h = h1data.h1data(fname)

In [ ]:
a = h1data.get_rgb_matrix(h)
b = h1data.get_rgb_matrix(h,True)

In [ ]:
px.imshow(a)

In [ ]:
px.imshow(b)

In [ ]:
from PIL import Image

# Create overlay image
wl = h.wavelengths

def is_water(spectra,wl,treshold = 2.4):
    #Empirical super simple water classifier
    C1 = np.argmin(abs(wl-460))
    C2 = np.argmin(abs(wl-650))
    return spectra[C1]/spectra[C2] > treshold

def get_is_water_map(cube,wl):
    C1 = np.argmin(abs(wl-460))
    C2 = np.argmin(abs(wl-650))
    ret = np.zeros([cube.shape[0], cube.shape[1]])
    for xx in range(cube.shape[0]):
        for yy in range(cube.shape[1]):
            spectra = cube[xx,yy,:]
            ret[xx,yy] = spectra[C1]/spectra[C2]
    return ret
    

def oc6_pace(cube,wl, treshold = 1):
    # From https://doi.org/10.1016/j.rse.2019.04.021
    
    result = np.zeros([cube.shape[0],cube.shape[1]])
    
    # Iitialize coefficients
    c = np.array([
        0.94297, -3.18493, 2.33682, -1.23923, 0.18697
    ])
    
    def get_chl_a(spectra, wl):
        # Find blue
        MBR_b = np.max([
            spectra[np.argmin(abs(wl-443))],
            spectra[np.argmin(abs(wl-490))],
            spectra[np.argmin(abs(wl-510))],
        ])
    
        # Find Green(red) 
        MBR_g = np.mean([
            spectra[np.argmin(abs(wl-555))],
            spectra[np.argmin(abs(wl-678))],
        ])
        
        X = MBR_b / MBR_g
        
        log_res = 0
        for i in range(len(c)):
            log_res += c[i]*X**i
        
        return 10**(log_res)
    
    # Iterate through cube
    for xx in range(b.shape[0]):
        for yy in range(b.shape[1]):
            spectra = cube[xx,yy,:]
                
            if not is_water(spectra,wl):
                result[xx,yy] = 0;
            else:
                r = get_chl_a(spectra,wl)
                if r > treshold:
                    result[xx,yy] = r
                else:
                    result[xx,yy] = 0
    return result

In [ ]:
r = get_is_water_map(h.l1a_cube,wl)
px.imshow(r)

In [ ]:
result = oc6_pace(h.l1a_cube,wl, treshold = 0)

In [ ]:
temp = np.zeros([b.shape[0], b.shape[1], 3])
temp[:,:,1] = result
alpha = 255*0.81
c = b + alpha*temp

In [ ]:
Image.fromarray(np.asarray(c*255,dtype=np.uint8)).save("taj_chl.png")

In [ ]:
Image.fromarray(np.asarray(b*255,dtype=np.uint8)).save("taj_before.png")